In [1]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en milisegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [2]:
def ping(host): 

    # Diccionario de códigos ICMP para respuestas de tipo 3 (Destino inalcanzable)
    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación requerida y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_ms": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")

    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:

        icmp_layer = response.getlayer(ICMP)

        code = icmp_layer.code

        rtt_ms = (fin - inicio) * 1000 # RTT en milisegundos
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl
        data["response_type"] = response.type

        if response.type == 3: 
            print(f"Destino {host} inalcanzable: " + codigos[code])
        
        elif response.type == 0: # Si la respuesta es echo reply
            print(f"Echo-reply recibida de {host}")

        else:
            print(f"Respuesta ICMP recibida de tipo {response.type}, código {icmp_layer.code}")

    else:
        print(f"No se recibió respuesta de {host}")

    return data

In [3]:
def ping_multiple(host, cantidad_paquetes):

    res = dict()
    sent_packets = cantidad_paquetes
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for i in range(cantidad_paquetes):

        resultado = ping(host)

        # print(resultado) # podemos descomentar para ver el resultado de cada ping

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
            lista_rtts.append(resultado["rtt_ms"])

        else:
            lost_packets += 1 # Si el destino es inalcanzable o no se recibe respuesta, el ping se considera perdido

    res["host"] = host
    res["Paquetes enviados"] = sent_packets
    res["Paquetes recibidos"] = received_packets
    res["Paquetes perdidos"] = lost_packets
    res["Porcentaje de paquetes perdidos"] = (lost_packets / sent_packets) 
    res["RTT total"] = total_rtt if received_packets > 0 else None
    res["RTT mínimo"] = min_rtt if min_rtt != float('inf') else None
    res["RTT máximo"] = max_rtt if max_rtt != float('-inf') else None
    res["RTT promedio"] = total_rtt / received_packets if received_packets > 0 else None
    res["Desvió estándar del RTT promedio"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

Contactamos a 5 universidades de continentes distintos. También contactamos a UTDT para ver las diferencias.

In [4]:
ping_multiple('utdt.edu.ar', 5)


Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar


{'host': 'utdt.edu.ar',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 96.04215621948242,
 'RTT mínimo': 14.534473419189453,
 'RTT máximo': 33.364057540893555,
 'RTT promedio': 19.208431243896484,
 'Desvió estándar del RTT promedio': 8.016241061224962}

In [5]:
ping_multiple('mit.edu', 5)

Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu


{'host': 'mit.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 232.3911190032959,
 'RTT mínimo': 42.084693908691406,
 'RTT máximo': 53.804636001586914,
 'RTT promedio': 46.47822380065918,
 'Desvió estándar del RTT promedio': 4.650285959168177}

In [6]:
ping_multiple('ie.edu', 5)

Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu


{'host': 'ie.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 84.21874046325684,
 'RTT mínimo': 9.094953536987305,
 'RTT máximo': 29.72102165222168,
 'RTT promedio': 16.843748092651367,
 'Desvió estándar del RTT promedio': 7.757640413584808}

In [7]:
ping_multiple('sydney.edu.au', 5)

Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au


{'host': 'sydney.edu.au',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 1613.8112545013428,
 'RTT mínimo': 318.9244270324707,
 'RTT máximo': 331.76231384277344,
 'RTT promedio': 322.76225090026855,
 'Desvió estándar del RTT promedio': 5.229950366904474}

In [8]:
ping_multiple('cuhk.edu.cn', 5)

Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn


{'host': 'cuhk.edu.cn',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 1670.057773590088,
 'RTT mínimo': 314.96667861938477,
 'RTT máximo': 383.12768936157227,
 'RTT promedio': 334.0115547180176,
 'Desvió estándar del RTT promedio': 28.2241186072276}

In [13]:
ping_multiple('uniben.edu', 5)

Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu


{'host': 'uniben.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 338.8545513153076,
 'RTT mínimo': 29.7086238861084,
 'RTT máximo': 118.79682540893555,
 'RTT promedio': 67.77091026306152,
 'Desvió estándar del RTT promedio': 38.08522442910583}